In [1]:
from ipycanvas import Canvas, hold_canvas
from Klasser.celle import *
from Klasser.busk import *
import random
from time import sleep

background_color = "#EEEEEE"

# visual size of tiles
tile_size = 15
visual_space = 2
rect_size = tile_size - visual_space

# amount of tiles in each direction
tiles_hori = 15
tiles_vert = 15

antal_celler = 1
start_energi = 10
sight_range = 3

antal_busker = 2

# Contains objects inhabiting the world
entities = []

# Create empty world, to contain references to cells
verd = []
for x in range(tiles_hori):
    row = []
    for y in range(tiles_vert):
        row.append(None)
    verd.append(row)

# Populate world
celler_igjen = antal_celler
busker_igjen = antal_busker
while celler_igjen > 0 or busker_igjen > 0:
    x = random.randint(0, tiles_hori - 1)
    y = random.randint(0, tiles_vert - 1)
    while verd[x][y] != None:
        x = random.randint(0, tiles_hori - 1)
        y = random.randint(0, tiles_vert - 1)
    if celler_igjen > 0:
        new_cell = Celle(x, y, start_energi, sight_range, tiles_hori, tiles_vert)
        entities.append(new_cell)
        verd[x][y] = len(entities) - 1
        celler_igjen -= 1
    else:
        new_busk = Busk(x, y, 0, 5)
        entities.append(new_busk)
        verd[x][y] = len(entities) - 1
        busker_igjen -= 1


# Canvas size
width = tile_size * tiles_hori
height = tile_size * tiles_vert

canvas = Canvas(height = height, width = width)
display(canvas)
vision_canvas = Canvas(height = height, width = width)
display(vision_canvas)

def draw_world(canvas):
    # Draw grid
    canvas.fill_style = background_color
    for x in range(tiles_hori):
        for y in range(tiles_vert):
            canvas.fill_rect(x * tile_size, y * tile_size, rect_size, rect_size)

def draw_entities(world, _canvas):
    with hold_canvas():
        x = 0
        for col in world:
            y = 0
            for tile in col:
                _canvas.fill_style = background_color
                if tile != None:
                    content = entities[tile]
                    if type(content) == Busk:
                        _canvas.fill_style = "green"
                    elif type(content) == Celle:
                        _canvas.fill_style = "blue"
                _canvas.fill_rect(x * tile_size, y * tile_size, rect_size, rect_size)
                y += 1
            x += 1

def rollover(list, index, radius):
    if index - radius < 0 or index + radius >= len(list) - 1:
        return list[index - radius:] + list[:(index + radius + 1) % len(list)]
    else:
        return list[index - radius : index + radius + 1]

def get_nabolag(pos_x, pos_y, radius):
    nabolag = []
    nabolag_rad = []
    for y in range(pos_y - radius, pos_y + radius + 1):
        for x in range(pos_x - radius, pos_x + radius + 1):
            nabolag_rad.append(verd[x % tiles_hori][y % tiles_vert])
        nabolag.append(nabolag_rad)
        nabolag_rad = []
    return nabolag

def draw_nabolag(nabolag, entity):
    with hold_canvas():
        x = 0
        for col in nabolag:
            y = 0
            for tile in col:
                vision_canvas.fill_style = background_color
                if tile != None:
                    content = entities[tile]
                    if type(content) == Busk:
                        vision_canvas.fill_style = "green"
                    elif type(content) == Celle:
                        vision_canvas.fill_style = "blue"
                vision_canvas.fill_rect(y * tile_size, x * tile_size, rect_size, rect_size)
                y += 1
            x += 1
        vision_canvas.fill_style = "tomato"
        vision_canvas.fill_rect(entity.mål_x * tile_size, entity.mål_y * tile_size, rect_size, rect_size)

#Løkke som repeterast til programmet er ferdig
while True:
    for j in range(len(entities)):
        entity = entities[j]
        if type(entity) == Busk:
            entity.veks()
        elif type(entity) == Celle:
            nabolag = get_nabolag(entity.pos_x, entity.pos_y, entity.sight_range)
            entity.finn_mål(nabolag, entities)
            draw_nabolag(nabolag, entity)
            beveg_x, beveg_y, dir_x, dir_y = entity.beveg()
            # Move the cell towards its target if there isnt anything in the way
            if verd[beveg_x % tiles_hori][beveg_y % tiles_vert] == None:
                verd[entity.pos_x][entity.pos_y] = None
                entity.update_pos(beveg_x % tiles_hori, beveg_y % tiles_vert, dir_x, dir_y)
                verd[beveg_x % tiles_hori][beveg_y % tiles_vert] = j
            else:
                entity.har_mål = False

    # Draw everything
    with hold_canvas():
        draw_world(canvas)
        for entity in entities:
            entity.tegn(canvas, tile_size, rect_size)
            if type(entity) == Celle:
                nabolag = get_nabolag(entity.pos_x, entity.pos_y, entity.sight_range)
                #draw_nabolag(nabolag, entity)
    
    sleep(0.3)
    #input("press enter to continue")

Canvas(height=225, width=225)

Canvas(height=225, width=225)

0
11


KeyboardInterrupt: 